In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/libs

Mounted at /gdrive
/gdrive/My Drive/libs


In [2]:
%matplotlib inline
import numpy as np
import scipy as sp
import scipy.sparse as sps
import matplotlib.pyplot as plt
import random
from scipy import stats
from scipy.optimize import fmin
import pandas as pd

In [3]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

data = pd.read_csv(filepath_or_buffer="data_train.csv",
                   sep=",")
data.columns = ["UserID", "ItemID", "Interaction"]

mapped_id, original_id = pd.factorize(data["UserID"].unique())
user_original_ID_to_index = pd.Series(mapped_id, index=original_id)

mapped_id, original_id = pd.factorize(data["ItemID"].unique())
item_original_ID_to_index = pd.Series(mapped_id, index=original_id)

data["UserID"] = data["UserID"].map(user_original_ID_to_index)
data["ItemID"] = data["ItemID"].map(item_original_ID_to_index)


URM_all = sps.coo_matrix((data["Interaction"].values,
                          (data["UserID"].values, data["ItemID"].values)))

URM_all.tocsr()

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

URM_all, URM_train_validation, URM_train, URM_validation, URM_test

(<12638x22222 sparse matrix of type '<class 'numpy.float64'>'
 	with 478730 stored elements in COOrdinate format>,
 <12638x22222 sparse matrix of type '<class 'numpy.float64'>'
 	with 382984 stored elements in Compressed Sparse Row format>,
 <12638x22222 sparse matrix of type '<class 'numpy.float64'>'
 	with 306387 stored elements in Compressed Sparse Row format>,
 <12638x22222 sparse matrix of type '<class 'numpy.float64'>'
 	with 76597 stored elements in Compressed Sparse Row format>,
 <12638x22222 sparse matrix of type '<class 'numpy.float64'>'
 	with 95746 stored elements in Compressed Sparse Row format>)

In [18]:
from Recommenders.BaseRecommender import BaseRecommender

class ScoresHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    Hybrid of two prediction scores R = R1*alpha + R2*(1-alpha)

    """

    RECOMMENDER_NAME = "ScoresHybridRecommender"

    '''def __init__(self, URM_train, alpha_R, beta, topK_R, alpha_S, l1_ratio, topK_S):
        super(ScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = RP3betaRecommender(URM_train)
        self.recommender_1.fit(alpha = alpha_R, beta =beta, topK = topK_R,implicit = False, normalize_similarity = True)

        self.recommender_2 = SLIMElasticNetRecommender(URM_train)
        self.recommender_2.fit(alpha = alpha_S, l1_ratio = l1_ratio, topK = topK_S)'''

    def __init__(self, URM_train, recommender1, recommeder2):
          super(ScoresHybridRecommender, self).__init__(URM_train)

          self.URM_train = sps.csr_matrix(URM_train)
          self.recommender_1 = recommender1
          self.recommender_2 = recommeder2


    def fit(self, alpha = 0.5):
        self.alpha = alpha


    def _compute_item_score(self, user_id_array, items_to_compute):

        # In a simple extension this could be a loop over a list of pretrained recommender objects
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array)

        item_weights = item_weights_1*self.alpha + item_weights_2*(1-self.alpha)

        return item_weights

In [20]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

RP3beta = RP3betaRecommender(URM_train)
RP3beta.fit(alpha = 0.31583032356728813, beta=0.1936831182299878, topK=29, implicit = False, normalize_similarity = True )

RP3betaRecommender: URM Detected 444 ( 3.5%) users with no interactions.
RP3betaRecommender: URM Detected 338 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 5494.24 column/sec. Elapsed time 4.04 sec


In [9]:
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

SLIM_MSE = SLIMElasticNetRecommender(URM_train)
SLIM_MSE.fit(alpha = 0.0026695828861131796, l1_ratio=9.7058223865738e-05, topK=237)

SLIMElasticNetRecommender: URM Detected 444 ( 3.5%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 338 ( 1.5%) items with no interactions.
SLIMElasticNetRecommender: Processed 12917 (58.1%) in 5.00 min. Items per second: 43.05
SLIMElasticNetRecommender: Processed 22222 (100.0%) in 8.70 min. Items per second: 42.58


#Try Hyperparameter tuning with Optuna

In [10]:
 pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 11.7 MB/s eta 0:00:00


In [11]:
from Evaluation.Evaluator import EvaluatorHoldout


evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 2608 (20.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2171 (17.2%) Users that have less than 1 test interactions


In [21]:
import optuna
import pandas as pd
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

#"alpha": Real(0.1,0.2),    "beta" : Real(0.1,0.2),    "topK" : Integer(10,50)


def objective_function(optuna_trial):


    recommender_instance = ScoresHybridRecommender(URM_train, RP3beta, SLIM_MSE
                                                   #alpha_S = optuna_trial.suggest_float("alpha_S", 0.001, 1.0),
                                                   #alpha_R = optuna_trial.suggest_float("alpha_R", 0.001, 1.0),
                                                   #beta = optuna_trial.suggest_float("beta", 0.001, 1.0),
                                                   #topK_S = optuna_trial.suggest_int("topK_S", 5, 300),
                                                   #topK_R = optuna_trial.suggest_int("topK_R", 5, 100),
                                                   #l1_ratio = optuna_trial.suggest_float("l1_ratio", 1e-6, 1e-4),
                                                   )
    recommender_instance.fit(alpha = optuna_trial.suggest_float("alpha", 0.001, 1.0),
                            )

    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)

    return result_df.loc[10]["MAP"]

In [22]:
class SaveResults(object):

    def __init__(self):
        self.results_df = pd.DataFrame(columns = ["result"])

    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]

        self.results_df = self.results_df.append(hyperparam_dict, ignore_index=True)

In [23]:
optuna_study = optuna.create_study(direction="maximize")

save_results = SaveResults()

optuna_study.optimize(objective_function,
                      callbacks=[save_results],
                      n_trials = 100)

[I 2023-12-24 16:56:02,181] A new study created in memory with name: no-name-d5581c4d-b820-4841-8402-8b545ec744fa


ScoresHybridRecommender: URM Detected 444 ( 3.5%) users with no interactions.
ScoresHybridRecommender: URM Detected 338 ( 1.5%) items with no interactions.
EvaluatorHoldout: Processed 10030 (100.0%) in 10.66 sec. Users per second: 941


[I 2023-12-24 16:56:12,878] Trial 0 finished with value: 0.03077180759309407 and parameters: {'alpha': 0.5459566664848782}. Best is trial 0 with value: 0.03077180759309407.


ScoresHybridRecommender: URM Detected 444 ( 3.5%) users with no interactions.
ScoresHybridRecommender: URM Detected 338 ( 1.5%) items with no interactions.


<ipython-input-22-650f32a8104f>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.results_df = self.results_df.append(hyperparam_dict, ignore_index=True)


...

[I 2023-12-24 17:08:06,861] Trial 98 finished with value: 0.030733185364541536 and parameters: {'alpha': 0.31928822060154616}. Best is trial 57 with value: 0.03083431056038257.


ScoresHybridRecommender: URM Detected 444 ( 3.5%) users with no interactions.
ScoresHybridRecommender: URM Detected 338 ( 1.5%) items with no interactions.


<ipython-input-22-650f32a8104f>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.results_df = self.results_df.append(hyperparam_dict, ignore_index=True)


EvaluatorHoldout: Processed 10030 (100.0%) in 7.00 sec. Users per second: 1432


[I 2023-12-24 17:08:13,903] Trial 99 finished with value: 0.03079564876798201 and parameters: {'alpha': 0.40435764180615824}. Best is trial 57 with value: 0.03083431056038257.
<ipython-input-22-650f32a8104f>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.results_df = self.results_df.append(hyperparam_dict, ignore_index=True)


In [24]:
pruned_trials = [t for t in optuna_study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in optuna_study.trials if t.state == optuna.trial.TrialState.COMPLETE]

print("Study statistics: ")
print("  Number of finished trials: ", len(optuna_study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
print("  Value Validation: ", optuna_study.best_trial.value)

Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  0
  Number of complete trials:  100
Best trial:
  Value Validation:  0.03083431056038257


In [25]:
optuna_study.best_trial

FrozenTrial(number=57, state=TrialState.COMPLETE, values=[0.03083431056038257], datetime_start=datetime.datetime(2023, 12, 24, 17, 2, 57, 645601), datetime_complete=datetime.datetime(2023, 12, 24, 17, 3, 4, 956902), params={'alpha': 0.4360593316742256}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'alpha': FloatDistribution(high=1.0, log=False, low=0.001, step=None)}, trial_id=57, value=None)

In [26]:
optuna_study.best_trial.params

{'alpha': 0.4360593316742256}

In [27]:
save_results.results_df

,result,alpha
0,0.030772,0.545957
1,0.030342,0.826620
2,0.030362,0.139233
3,0.030792,0.404701
4,0.029863,0.984109
...,...,...
95,0.030828,0.432439
96,0.030819,0.426123
97,0.030766,0.375171
98,0.030733,0.319288


In [28]:
recommender_instance = (URM_train + URM_validation)
recommender_instance.fit(**optuna_study.best_trial.params)

result_df, _ = evaluator_test.evaluateRecommender(recommender_instance)
result_df

AttributeError: ignored

#Let's submit

RP3beta.fit(alpha = 0.30045048565736454, beta=0.15900639335692543, topK=34, implicit = False, normalize_similarity = True) + SLIM_MSE.fit(alpha = 0.0033868445794621033, l1_ratio=2.1304972208263965e-05, topK=36) + alpha = 0.5042400731098166. val:3.09473% submit 13.62%

RP3beta.fit(alpha = 0.30045048565736454, beta=0.15900639335692543, topK=34, implicit = False, normalize_similarity = True) + SLIM_MSE.fit(alpha = 0.0026695828861131796, l1_ratio=9.7058223865738e-05, topK=237) + alpha:0.42280199553592035 val: 3.09616% submit:13.801%

RP3beta.fit(alpha = 0.3388784009010535, beta=0.177797903368052, topK=30, implicit = False, normalize_similarity = True) + SLIM_MSE.fit(alpha = 0.0026695828861131796, l1_ratio=9.7058223865738e-05, topK=237) + alpha = 0.5616855740532924 val: 3.082% submit: 13.766%

RP3beta.fit(alpha = 0.30045048565736454, beta=0.15900639335692543, topK=34, implicit = False, normalize_similarity = True) +
SLIM_MSE.fit(alpha = 0.0030080355721395845, l1_ratio=5.495490764642386e-06, topK=207) +  alpha:0.4451229997552284 + val:3.034% submit:13.8%

RP3beta.fit(alpha = 0.32572640698412575, beta=0.18217576354665624, topK=31, implicit = False, normalize_similarity = True) + SLIM_MSE.fit(alpha = 0.0026695828861131796, l1_ratio=9.7058223865738e-05, topK=237) + alpha:0.4420834976325743 val3.146% submit: 13.82%

RP3beta.fit(alpha = 0.31583032356728813, beta=0.1936831182299878, topK=29, implicit = False, normalize_similarity = True ) + SLIM_MSE.fit(alpha = 0.0026695828861131796, l1_ratio=9.7058223865738e-05, topK=237) + alpha:0.4360593316742256 val:3.0834% submit:


In [29]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

RP3beta = RP3betaRecommender(URM_all)
RP3beta.fit(alpha = 0.31583032356728813, beta=0.1936831182299878, topK=29, implicit = False, normalize_similarity = True )

RP3betaRecommender: Similarity column 22222 (100.0%), 3919.18 column/sec. Elapsed time 5.67 sec


In [30]:
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

SLIM_MSE = SLIMElasticNetRecommender(URM_all)
SLIM_MSE.fit(alpha = 0.0026695828861131796, l1_ratio=9.7058223865738e-05, topK=237)

SLIMElasticNetRecommender: Processed 10610 (47.7%) in 5.00 min. Items per second: 35.36
SLIMElasticNetRecommender: Processed 22222 (100.0%) in 10.00 min. Items per second: 37.04


In [31]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender


In [32]:
#{'alpha_S': 0.0018039635745485064, 'alpha_R': 0.0034513779391265736, 'beta': 0.9111983209661509, 'topK_S': 187, 'topK_R': 93, 'l1_ratio': 9.371084323801284e-05, 'alpha': 0.2838202981991156}.

recommender = ScoresHybridRecommender(URM_all, RP3beta, SLIM_MSE)
recommender.fit(0.4360593316742256)

In [33]:
users_to_recommend_raw = pd.read_csv(filepath_or_buffer="/gdrive/My Drive/libs/data_target_users_test.csv",
                   sep=",",
                  dtype={"user_id": np.int32}).to_numpy()
users_to_recommend = []
for elem in users_to_recommend_raw:
  users_to_recommend.append(elem[0])

len(users_to_recommend)

10882

In [34]:
ratings = pd.read_csv("/gdrive/My Drive/libs/data_train2.csv",
                       sep=",",
                       names=["user_id", "item_id", "ratings"],
                       header=None,
                       dtype={"user_id": np.int64,
                               "item_id": np.int64,
                               "ratings": np.int64})
ratings.shape

(478730, 3)

In [35]:
def preprocess_data(ratings: pd.DataFrame):
    unique_users = ratings.user_id.unique()
    unique_items = ratings.item_id.unique()

    num_users, min_user_id, max_user_id = unique_users.size, unique_users.min(), unique_users.max()
    num_items, min_item_id, max_item_id = unique_items.size, unique_items.min(), unique_items.max()

    print(num_users, min_user_id, max_user_id)
    print(num_items, min_item_id, max_item_id)

    mapping_user_id = pd.DataFrame({"mapped_user_id": np.arange(num_users), "user_id": unique_users})
    mapping_item_id = pd.DataFrame({"mapped_item_id": np.arange(num_items), "item_id": unique_items})

    ratings = pd.merge(left=ratings,
                       right=mapping_user_id,
                       how="inner",
                       on="user_id")

    ratings = pd.merge(left=ratings,
                       right=mapping_item_id,
                       how="inner",
                       on="item_id")

    return ratings


In [36]:
ratings = preprocess_data(ratings)

12638 1 13024
22222 1 22347


In [37]:
users_ids_and_mappings = ratings[ratings.user_id.isin(users_to_recommend)][["user_id", "mapped_user_id"]].drop_duplicates()
items_ids_and_mappings = ratings[["item_id", "mapped_item_id"]].drop_duplicates()
users_ids_and_mappings

,user_id,mapped_user_id
0,1,0
1,2,1
2,26,24
3,36,34
4,41,39
...,...,...
469974,9966,9677
470088,11525,11196
471697,11994,11649
476929,10065,9773


In [38]:
def prepare_submission(ratings: pd.DataFrame, users_to_recommend: np.array, urm_train: sps.csr_matrix):
    users_ids_and_mappings = ratings[ratings.user_id.isin(users_to_recommend)][["user_id", "mapped_user_id"]].drop_duplicates()
    items_ids_and_mappings = ratings[["item_id", "mapped_item_id"]].drop_duplicates()

    mapping_to_item_id = dict(zip(ratings.mapped_item_id, ratings.item_id))


    recommendation_length = 10
    submission = []
    for idx, row in users_ids_and_mappings.iterrows():
        user_id = row.user_id
        mapped_user_id = row.mapped_user_id
        recommendations = recommender.recommend(mapped_user_id,recommendation_length)
        submission.append((user_id, [mapping_to_item_id[item_id] for item_id in recommendations]))

    users_recommended = [elem[0] for elem in submission]

    missing_users = []

    top10 = [2, 4, 1, 7, 3, 6, 8, 9, 15, 20]  #con remove seen = False
    for elem in users_to_recommend:
      if elem not in users_recommended:
        missing_users.append(elem)

    for i in range(len(missing_users)):
      missing_users[i] = (missing_users[i], [mapping_to_item_id[top10[elem]] for elem in range(10)])

    return submission, missing_users

In [39]:
URM_train = URM_all.tocsr()
submission, missing_users = prepare_submission(ratings, users_to_recommend, URM_all)
len(submission), len(missing_users)


(10661, 221)

In [40]:
submission = submission + missing_users
len(submission)

10882

In [41]:
submission.sort()

In [42]:
def write_submission(submissions):
    with open("./Hybrid_12.csv", "w") as f:
        f.write("user_id,item_list\n")
        for user_id, items in submissions:
          f.write(f"{user_id},{' '.join(str(item) for item in items)}\n")



In [43]:
write_submission(submission)